# https://www.kaggle.com/darkside92/detailed-examination-for-house-price-top-10

In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from scipy.stats import skew
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax
from scipy.stats import boxcox
from scipy.special import inv_boxcox
import seaborn as sb
import matplotlib.pyplot as plt
!pip install lightgbm
!pip install xgboost
import lightgbm as lgb
import xgboost as xgb
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV, KFold,GridSearchCV
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error
from sklearn.preprocessing import StandardScaler,RobustScaler,LabelEncoder,PowerTransformer
from sklearn.ensemble import GradientBoostingRegressor,StackingRegressor, RandomForestRegressor, ExtraTreesRegressor
from sklearn.linear_model import ElasticNet, Lasso, Ridge
from sklearn.model_selection import KFold, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeRegressor
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import PCA
import seaborn as sns  
import warnings
warnings.filterwarnings("ignore")         # for beautiful plots
from scipy import stats                 # to calculate mode, skew and ANOVA, etc.
import numpy as np # linear algebra
np.set_printoptions(np.inf)
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import norm, skew #for some statistics
plt.rcParams['figure.figsize'] = (10, 8)
# importing alll the necessary packages to use the various classification algorithms
from sklearn.linear_model import LogisticRegression  # for Logistic Regression algorithm
from sklearn.model_selection import train_test_split #to split the dataset for training and testing
from sklearn.neighbors import KNeighborsClassifier  # for K nearest neighbours
from sklearn import svm  #for Support Vector Machine (SVM) Algorithm
from sklearn import metrics #for checking the model accuracy
from sklearn.metrics import mean_squared_error #for checking the model accuracy
from sklearn.tree import DecisionTreeClassifier #for using Decision Tree Algoithm
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
np.set_printoptions(threshold=np.inf)
import xgboost as xgb
import lightgbm as lgb
import time
import pickle
import gc

In [126]:
pd.set_option('display.max_columns', 5000)
pd.set_option('display.max_rows', 5000)
train = pd.read_csv('../house price/train.csv')
test = pd.read_csv('../house price/test.csv')
sample_sub = pd.read_csv('../house price/sample_submission.csv')

In [127]:
train = pd.read_pickle('../house price/011data.pkl')[0:1433]
test = pd.read_pickle('../house price/011data.pkl')[1433:2919]

X = train.drop(['SalePrice','SalePrice_Log1p','Id'],axis=1)
y = train.SalePrice_Log1p
test = test.drop(['SalePrice','SalePrice_Log1p','Id'],axis=1)

def overfit_reducer(df):

    overfit = []
    for i in df.columns:
        counts = df[i].value_counts()
        zeros = counts.iloc[0]
        if zeros / len(df) * 100 > 99.9:
            overfit.append(i)
    overfit = list(overfit)
    return overfit

overfitted_features = overfit_reducer(X)
print(overfitted_features)
print(X.shape)
print(test.shape)
X.drop(overfitted_features, axis=1, inplace=True)
test.drop(overfitted_features, axis=1, inplace=True)
print('X.finalshape',X.shape)
print('test.finalshape',test.shape)

['PoolArea', 'PoolQC', 'exists_pool', 'Utilities_AllPub', 'Utilities_NoSeWa', 'Condition2_PosA', 'Condition2_PosN', 'Condition2_RRAe', 'Condition2_RRAn', 'RoofMatl_ClyTile', 'RoofMatl_Membran', 'RoofMatl_Metal', 'RoofMatl_Roll', 'Exterior1st_AsphShn', 'Exterior1st_CBlock', 'Exterior1st_ImStucc', 'Exterior2nd_CBlock', 'Exterior2nd_Other', 'Heating_Floor', 'Electrical_Mix', 'MiscFeature_TenC']
(1433, 239)
(1459, 239)
X.finalshape (1433, 218)
test.finalshape (1459, 218)


## データの標準化

In [128]:
std_scaler=StandardScaler()
rbst_scaler=RobustScaler()
power_transformer=PowerTransformer()
X_std = pd.DataFrame(std_scaler.fit_transform(X))
X_rbst = pd.DataFrame(rbst_scaler.fit_transform(X))
X_pwr = pd.DataFrame(power_transformer.fit_transform(X))

test_std = pd.DataFrame(std_scaler.transform(test))
test_rbst = pd.DataFrame(rbst_scaler.transform(test))
test_pwr = pd.DataFrame(power_transformer.transform(test))

In [129]:
y.isnull().value_counts()

False    1433
Name: SalePrice_Log1p, dtype: int64

## selected

In [130]:
s_train = pd.read_pickle('../house price/010data.pkl')[0:1433]
s_test = pd.read_pickle('../house price/010data.pkl')[1433:2919]

s_X = s_train.drop(['SalePrice','SalePrice_Log1p','Id'],axis=1)
s_y = s_train.SalePrice_Log1p
s_test = s_test.drop(['SalePrice','SalePrice_Log1p','Id'],axis=1)

def overfit_reducer(df):

    overfit = []
    for i in df.columns:
        counts = df[i].value_counts()
        zeros = counts.iloc[0]
        if zeros / len(df) * 100 > 99.9:
            overfit.append(i)
    overfit = list(overfit)
    return overfit

overfitted_features = overfit_reducer(s_X)
print(overfitted_features)
print(s_X.shape)
print(s_test.shape)
s_X.drop(overfitted_features, axis=1, inplace=True)
s_test.drop(overfitted_features, axis=1, inplace=True)
print('X.finalshape',s_X.shape)
print('test.finalshape',s_test.shape)

s_X_std = pd.DataFrame(std_scaler.fit_transform(s_X))
s_X_rbst = pd.DataFrame(rbst_scaler.fit_transform(s_X))
s_X_pwr = pd.DataFrame(power_transformer.fit_transform(s_X))

s_test_std = pd.DataFrame(std_scaler.transform(s_test))
s_test_rbst = pd.DataFrame(rbst_scaler.transform(s_test))
s_test_pwr = pd.DataFrame(power_transformer.transform(s_test))

['PoolArea', 'PoolQC', 'exists_pool', 'Utilities_AllPub', 'Utilities_NoSeWa', 'Condition2_PosA', 'Condition2_PosN', 'Condition2_RRAe', 'Condition2_RRAn', 'RoofMatl_ClyTile', 'RoofMatl_Membran', 'RoofMatl_Metal', 'RoofMatl_Roll', 'Exterior1st_AsphShn', 'Exterior1st_CBlock', 'Exterior1st_ImStucc', 'Exterior2nd_CBlock', 'Exterior2nd_Other', 'Heating_Floor', 'Electrical_Mix', 'MiscFeature_TenC']
(1433, 230)
(1459, 230)
X.finalshape (1433, 209)
test.finalshape (1459, 209)


## バリデーション設計と評価指標

In [131]:
def predict_cv(model, train_x, train_y, test_x):
    preds = []
    preds_test = []
    va_indexes = []
    
    kf = KFold(n_splits=4, shuffle=True, random_state=6785)
    
    # クロスバリデーションで学習・予測を行い、予測値とインデックスを保存する
    for i, (tr_idx, va_idx) in enumerate(kf.split(train_x)):
        tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
        tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]
        model.fit(tr_x, tr_y)
        tr_pred = model.predict(tr_x)
        pred = model.predict(va_x)
        preds.append(pred)
        pred_test = model.predict(test_x)
        preds_test.append(pred_test)
        va_indexes.append(va_idx)
        print('  score Train : {:.6f}' .format(np.sqrt(mean_squared_error(tr_y, tr_pred))), 
              '  score Valid : {:.6f}' .format(np.sqrt(mean_squared_error(va_y, pred)))) 
        
    # バリデーションデータに対する予測値を連結し、その後元の順番に並べなおす
    va_indexes = np.concatenate(va_indexes)
    preds = np.concatenate(preds, axis=0)
    order = np.argsort(va_indexes)
    pred_train = pd.DataFrame(preds[order])
    
    
    # テストデータに対する予測値の平均をとる
    preds_test = pd.DataFrame(np.mean(preds_test, axis=0))
    print('Score : {:.6f}' .format(np.sqrt(mean_squared_error(train_y, pred_train))))
    return pred_train, preds_test, model

## LightGBM

In [132]:
lgbm = lgb.LGBMRegressor(objective='regression',
                          num_leaves=2, 
                          learning_rate=0.035,
                          n_estimators=2177,
                          max_bin=200,
                          bagging_fraction=0.65,
                          bagging_freq=5, 
                          bagging_seed=7, 
                          feature_fraction=0.201, 
                          feature_fraction_seed=7,
                          n_jobs=-1)

lgbm_pred_train, lgbm_preds_test, lgbm = predict_cv(lgbm,  s_X_pwr, s_y, s_test_pwr)

  score Train : 0.093182   score Valid : 0.118831
  score Train : 0.093026   score Valid : 0.121786
  score Train : 0.095731   score Valid : 0.111337
  score Train : 0.092416   score Valid : 0.117907
Score : 0.117528


## Ridge回帰

In [133]:
ridge = Ridge(alpha=1)
ridge_pred_train, ridge_preds_test, ridge = predict_cv(ridge,  X_pwr, y, test_pwr)

  score Train : 0.083759   score Valid : 0.132442
  score Train : 0.086974   score Valid : 0.121943
  score Train : 0.091301   score Valid : 0.105050
  score Train : 0.086381   score Valid : 0.121313
Score : 0.120594


## Lasso回帰

In [134]:
lasso = Lasso(alpha=0.001)
lasso_pred_train, lasso_preds_test, lasso = predict_cv(lasso,  X_pwr, y, test_pwr)

  score Train : 0.085992   score Valid : 0.125102
  score Train : 0.089115   score Valid : 0.111125
  score Train : 0.093324   score Valid : 0.098694
  score Train : 0.089249   score Valid : 0.111079
Score : 0.111901


## ElasticNet

In [135]:
elastic = ElasticNet(alpha=0.01, l1_ratio=0.7)
elastic_pred_train, elastic_preds_test, elastic = predict_cv(elastic,  X_pwr, y, test_pwr)

  score Train : 0.099047   score Valid : 0.117792
  score Train : 0.103135   score Valid : 0.108866
  score Train : 0.104832   score Valid : 0.104982
  score Train : 0.102134   score Valid : 0.109472
Score : 0.110382


## Stacking

In [136]:
estimators = [('lgbm', lgbm),  
              ('lasso', lasso),   
              ('ridge', ridge),   
              ('elasticnet', elastic)]

stack = StackingRegressor(estimators=estimators, final_estimator=ExtraTreesRegressor(n_estimators=100), n_jobs=-1)
stack_pred_train, stack_preds_test, stack = predict_cv(stack,  X_pwr, y, test_pwr)

  score Train : 0.087720   score Valid : 0.128819
  score Train : 0.089898   score Valid : 0.121114
  score Train : 0.100249   score Valid : 0.107430
  score Train : 0.091263   score Valid : 0.113342
Score : 0.117960


In [137]:
print('Count columns of test: ',len(test.columns))
print('Count columns of X: ',len(X.columns))

Count columns of test:  218
Count columns of X:  218


## trainの予測値を見る

In [138]:
lgbm_pred_train.columns = ['SalePrice']
elastic_pred_train.columns = ['SalePrice']
ridge_pred_train.columns = ['SalePrice']
lasso_pred_train.columns = ['SalePrice']
stack_pred_train.columns = ['SalePrice']

In [139]:
print('  score LGBM Train : {:.6f}' .format(np.sqrt(mean_squared_error(y, lgbm_pred_train))))
print('  score eNetTrain : {:.6f}' .format(np.sqrt(mean_squared_error(y, elastic_pred_train))))
print('  score ridge Train : {:.6f}' .format(np.sqrt(mean_squared_error(y, ridge_pred_train))))
print('  score Lasso Train : {:.6f}' .format(np.sqrt(mean_squared_error(y, lasso_pred_train))))
print('  score Stack Train : {:.6f}' .format(np.sqrt(mean_squared_error(y, stack_pred_train))))

  score LGBM Train : 0.117528
  score eNetTrain : 0.110382
  score ridge Train : 0.120594
  score Lasso Train : 0.111901
  score Stack Train : 0.117960


In [140]:
best_pram_score = []
scores = []
for i in np.arange(0, 1, 0.1):
    for j in np.arange(0, 1-i, 0.1):
        for k in np.arange(0, 1-(i+j), 0.1):
            for l in np.arange(0, 1-(i+j+k), 0.1):
                m = 1-(i+j+k+l)
                train_pred = i*lgbm_pred_train + j*elastic_pred_train + k*ridge_pred_train + l*lasso_pred_train + m*stack_pred_train
                score = np.sqrt(mean_squared_error(y, train_pred))
                best_pram_score.append([score, train_pred, i,j,k,l,m])

In [141]:
best_param_score = pd.DataFrame(best_pram_score)
best_param_score.columns = ['score', 'df', 'LGBM', 'eNet', 'Ridge', 'Lasso', 'Stack']
best_param = best_param_score[best_param_score['score'] == best_param_score['score'].min()]
i,j,k,l,m = best_param.iat[0, 2], best_param.iat[0, 3], best_param.iat[0, 4], best_param.iat[0, 5], best_param.iat[0,6]
best_param

,score,df,LGBM,eNet,Ridge,Lasso,Stack
495,0.107518,SalePrice 0 12.237495 1 12.16658...,0.2,0.4,0.0,0.3,0.1


In [142]:
best_param_score.sort_values(by='score').head()

,score,df,LGBM,eNet,Ridge,Lasso,Stack
495,0.107518,SalePrice 0 12.237495 1 12.16658...,0.2,0.4,0.0,0.3,0.1
583,0.107564,SalePrice 0 12.237315 1 12.16056...,0.3,0.3,0.0,0.3,0.1
480,0.107572,SalePrice 0 12.238115 1 12.17003...,0.2,0.3,0.0,0.4,0.1
505,0.107604,SalePrice 0 12.233977 1 12.16567...,0.2,0.5,0.0,0.3,0.0
592,0.107658,SalePrice 0 12.236695 1 12.15711...,0.3,0.4,0.0,0.2,0.1


## testの予測値を見る

In [143]:
lgbm_preds_test.columns = ['SalePrice']
elastic_preds_test.columns = ['SalePrice']
ridge_preds_test.columns = ['SalePrice']
lasso_preds_test.columns = ['SalePrice']
stack_preds_test.columns = ['SalePrice']

lgbm_preds_test['SalePrice'] = np.floor(np.expm1(lgbm_preds_test['SalePrice']))
elastic_preds_test['SalePrice'] = np.floor(np.expm1(elastic_preds_test['SalePrice']))
ridge_preds_test['SalePrice'] = np.floor(np.expm1(ridge_preds_test['SalePrice']))
lasso_preds_test['SalePrice'] = np.floor(np.expm1(lasso_preds_test['SalePrice']))
stack_preds_test['SalePrice'] = np.floor(np.expm1(stack_preds_test['SalePrice']))

In [144]:
lgbm_preds_test.tail()

,SalePrice
1454,87051.0
1455,84063.0
1456,168658.0
1457,130819.0
1458,230952.0


In [145]:
elastic_preds_test.tail()

,SalePrice
1454,85104.0
1455,82710.0
1456,172785.0
1457,122790.0
1458,229746.0


In [146]:
ridge_preds_test.tail()

,SalePrice
1454,85654.0
1455,83240.0
1456,176817.0
1457,128221.0
1458,235007.0


In [147]:
lasso_preds_test.tail()

,SalePrice
1454,84651.0
1455,81982.0
1456,175317.0
1457,124646.0
1458,233291.0


In [148]:
stack_preds_test.tail()

,SalePrice
1454,86982.0
1455,82947.0
1456,171486.0
1457,126083.0
1458,231867.0


## 加重平均

In [149]:
final_pred = (lgbm_preds_test*i) + (elastic_preds_test*j) + (ridge_preds_test*k) + (ridge_preds_test*l) + (stack_preds_test*m)
final_pred.tail()

,SalePrice
1454,85846.2
1455,83163.3
1456,173039.3
1457,126354.4
1458,231777.6


In [150]:
sample_sub['SalePrice'] = final_pred
sample_sub.tail()

,Id,SalePrice
1454,2915,85846.2
1455,2916,83163.3
1456,2917,173039.3
1457,2918,126354.4
1458,2919,231777.6


In [151]:
sample_sub.to_csv('submission_.12327_ratio3302_0805_2007.csv',index=False)
#0.12275

 0.12414 ENet*0.5 LGBM*0.5 cols=230
 
 0.13137 Enet Only cols=230
 
 0.12423 LGBM Only cols=230
 
 0.13972 Enet Only cols=50 submissionに対しては過学習か 
 
 0.12772 ENet*0.5 LGBM*0.5 cols=50, 230
 
 0.12465 ENet*0.2 LGBM*0.8 cols=50, 230
 


# 最後を加重平均ではなくスタッキングにしてみる
モデル同士の相関関係を無視しているのであまり良くない気がする

In [152]:
estimators = [('lgbm', lgbm),  
              ('elasticnet', elastic),
              ('stacking', stack)]

f_stack = StackingRegressor(estimators=estimators,final_estimator=ExtraTreesRegressor(n_estimators=100), n_jobs=-1)
f_stack_pred_train, f_stack_preds_test, f_stack = predict_cv(f_stack,  X_pwr, y, test_pwr)

  score Train : 0.097123   score Valid : 0.122230
  score Train : 0.095923   score Valid : 0.120737
  score Train : 0.102420   score Valid : 0.107560
  score Train : 0.096976   score Valid : 0.112285
Score : 0.115865


In [153]:
f_stack_preds_test.columns = ['SalePrice']
f_stack_preds_test['SalePrice'] = np.floor(np.expm1(f_stack_preds_test['SalePrice']))

In [155]:
sample_sub['SalePrice'] = f_stack_preds_test['SalePrice']
sample_sub.tail()

,Id,SalePrice
1454,2915,89217.0
1455,2916,89336.0
1456,2917,167470.0
1457,2918,124184.0
1458,2919,228481.0


In [156]:
sample_sub.to_csv('submission_.finalstack_0805_2028.csv',index=False)